In [5]:
import os
import time
import datetime
import requests
import pandas as pd
import pyarrow
import numpy as np
from prefect import flow, task # Prefect flow and task decorators
import time
from datetime import datetime, timezone
from zoneinfo import ZoneInfo

In [13]:
#read firms_output parquet
df_firms = pd.read_parquet("firms_outout", engine="pyarrow")
df_firms.info()
df_firms.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2998 entries, 0 to 2997
Data columns (total 22 columns):
 #   Column           Non-Null Count  Dtype                       
---  ------           --------------  -----                       
 0   country_id       2998 non-null   object                      
 1   latitude         2998 non-null   float64                     
 2   longitude        2998 non-null   float64                     
 3   brightness       2998 non-null   float64                     
 4   scan             2998 non-null   float64                     
 5   track            2998 non-null   float64                     
 6   acq_date         2998 non-null   object                      
 7   acq_time         2998 non-null   object                      
 8   satellite        2998 non-null   object                      
 9   instrument       2998 non-null   object                      
 10  confidence       2998 non-null   int64                       
 11  version          

,country_id,latitude,longitude,brightness,scan,track,acq_date,acq_time,satellite,instrument,...,bright_t31,frp,daynight,acq_datetime,acq_datetime_th,acq_year,acq_month,acq_day,acq_hour,acq_minute
0,THA,19.45405,100.56052,310.83,2.73,1.58,2025-03-28,03:28:00,Terra,MODIS,...,297.41,28.27,D,2025-03-28 03:28:00,2025-03-28 10:28:00+07:00,2025,3,28,10,28
1,THA,19.45886,100.53445,316.00,2.71,1.58,2025-03-28,03:28:00,Terra,MODIS,...,298.49,43.84,D,2025-03-28 03:28:00,2025-03-28 10:28:00+07:00,2025,3,28,10,28
2,THA,19.46471,100.54360,315.08,2.71,1.58,2025-03-28,03:28:00,Terra,MODIS,...,298.54,39.24,D,2025-03-28 03:28:00,2025-03-28 10:28:00+07:00,2025,3,28,10,28
3,THA,19.47333,98.03783,309.05,1.75,1.30,2025-03-28,03:28:00,Terra,MODIS,...,296.34,12.68,D,2025-03-28 03:28:00,2025-03-28 10:28:00+07:00,2025,3,28,10,28
4,THA,14.07963,100.65222,321.68,3.57,1.77,2025-03-28,03:30:00,Terra,MODIS,...,297.11,64.02,D,2025-03-28 03:30:00,2025-03-28 10:30:00+07:00,2025,3,28,10,30


In [14]:
#read weather_output parquet
df_weather = pd.read_parquet("weather_outout", engine="pyarrow")
df_weather.info()
df_weather.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3388 entries, 0 to 3387
Data columns (total 39 columns):
 #   Column               Non-Null Count  Dtype                       
---  ------               --------------  -----                       
 0   coord.lon            3388 non-null   float64                     
 1   coord.lat            3388 non-null   float64                     
 2   weather.id           3388 non-null   int64                       
 3   weather.main         3388 non-null   object                      
 4   weather.description  3388 non-null   object                      
 5   weather.icon         3388 non-null   object                      
 6   base                 3388 non-null   object                      
 7   main.temp            3388 non-null   float64                     
 8   main.feels_like      3388 non-null   float64                     
 9   main.pressure        3388 non-null   int64                       
 10  main.humidity        3388 non-null  

,coord.lon,coord.lat,weather.id,weather.main,weather.description,weather.icon,base,main.temp,main.feels_like,main.pressure,...,sys.sunset,timezone,id,name,cod,acq_year,acq_month,acq_day,acq_hour,acq_minute
0,100.5241,13.7279,803,Clouds,broken clouds,04n,stations,302.50,307.95,1004,...,1743161353,25200,1619278,Bangrak,200,2025,3,28,1,6
1,98.9063,8.0863,804,Clouds,overcast clouds,04n,stations,298.67,299.47,1007,...,1743161665,25200,1152633,Krabi,200,2025,3,28,1,6
2,99.5328,14.0228,802,Clouds,scattered clouds,03n,stations,301.23,302.61,1004,...,1743161595,25200,1157835,Ban Don Rak,200,2025,3,28,1,6
3,103.5059,16.4314,804,Clouds,overcast clouds,04n,stations,304.73,305.54,1002,...,1743160676,25200,1610469,Kalasin,200,2025,3,28,1,6
4,99.5227,16.4828,802,Clouds,scattered clouds,03n,stations,299.77,299.77,1003,...,1743161633,25200,1153090,Kamphaeng Phet,200,2025,3,28,1,6


In [15]:
df_weather = df_weather.rename(columns={'coord.lat': 'latitude', 'coord.lon': 'longitude'})

In [9]:
import pandas as pd
import numpy as np

from sklearn.neighbors import BallTree
from io import StringIO

import matplotlib.pyplot as plt
import contextily as ctx

In [16]:
# Convert coordinates to radians
firms_coords = np.deg2rad(df_firms[['latitude', 'longitude']])
weather_coords = np.deg2rad(df_weather[['latitude', 'longitude']])

# Create a BallTree for nearest neighbor search
tree = BallTree(weather_coords, metric='haversine')

# Find the nearest neighbors in OSM data
distances, indices = tree.query(firms_coords, k=1)

test_df = pd.DataFrame()
# Add the results to the DataFrame
df_firms['nearest_index'] = indices.flatten()
df_firms['distance_meters'] = distances.flatten() * 6371000 # Earth's radius in meters

# Extract and assign the matched OSM node details
df_firms[[
    'near_lat',
    'near_lon',
]] = df_weather.iloc[df_firms['nearest_index']][[
    'latitude',
    'longitude',
]].values

# test_df['node_id'] = test_df['node_id'].apply(lambda x: f'{int(x)}')
# test_df = test_df.drop('nearest_index', axis=1)

df_firms.head(5)

,country_id,latitude,longitude,brightness,scan,track,acq_date,acq_time,satellite,instrument,...,acq_datetime_th,acq_year,acq_month,acq_day,acq_hour,acq_minute,nearest_index,distance_meters,near_lat,near_lon
0,THA,19.45405,100.56052,310.83,2.73,1.58,2025-03-28,03:28:00,Terra,MODIS,...,2025-03-28 10:28:00+07:00,2025,3,28,10,28,651,46033.117171,19.2149,100.2024
1,THA,19.45886,100.53445,316.00,2.71,1.58,2025-03-28,03:28:00,Terra,MODIS,...,2025-03-28 10:28:00+07:00,2025,3,28,10,28,651,44154.981405,19.2149,100.2024
2,THA,19.46471,100.54360,315.08,2.71,1.58,2025-03-28,03:28:00,Terra,MODIS,...,2025-03-28 10:28:00+07:00,2025,3,28,10,28,651,45311.666783,19.2149,100.2024
3,THA,19.47333,98.03783,309.05,1.75,1.30,2025-03-28,03:28:00,Terra,MODIS,...,2025-03-28 10:28:00+07:00,2025,3,28,10,28,1123,20509.954218,19.3020,97.9654
4,THA,14.07963,100.65222,321.68,3.57,1.77,2025-03-28,03:30:00,Terra,MODIS,...,2025-03-28 10:30:00+07:00,2025,3,28,10,30,386,14352.531111,14.0204,100.5340


In [17]:
df_firms = df_firms.sort_values('acq_datetime_th')
df_weather = df_weather.sort_values('dt')

In [18]:
dfmerge = pd.merge_asof(df_firms, df_weather,
...                         left_on='acq_datetime_th' , right_on='dt',
                            left_by=['near_lat','near_lon'], right_by=['latitude','longitude'],
                            direction='nearest')

In [19]:
dfmerge.info()
dfmerge.head(5)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2998 entries, 0 to 2997
Data columns (total 65 columns):
 #   Column               Non-Null Count  Dtype                       
---  ------               --------------  -----                       
 0   country_id           2998 non-null   object                      
 1   latitude_x           2998 non-null   float64                     
 2   longitude_x          2998 non-null   float64                     
 3   brightness           2998 non-null   float64                     
 4   scan                 2998 non-null   float64                     
 5   track                2998 non-null   float64                     
 6   acq_date_x           2998 non-null   object                      
 7   acq_time_x           2998 non-null   object                      
 8   satellite            2998 non-null   object                      
 9   instrument           2998 non-null   object                      
 10  confidence           2998 non-null  

,country_id,latitude_x,longitude_x,brightness,scan,track,acq_date_x,acq_time_x,satellite,instrument,...,sys.sunset,timezone,id,name,cod,acq_year_y,acq_month_y,acq_day_y,acq_hour_y,acq_minute_y
0,THA,16.25453,102.07751,310.48,3.16,1.68,2025-03-27,19:21:00,Aqua,MODIS,...,1743161021,25200,1611407,Chaiyaphum,200,2025,3,28,1,6
1,THA,17.09465,101.71175,303.78,3.47,1.75,2025-03-27,19:21:00,Aqua,MODIS,...,1743160947,25200,1608269,Nong Bua Lamphu,200,2025,3,28,1,6
2,THA,16.25340,102.08438,309.56,3.16,1.68,2025-03-27,19:21:00,Aqua,MODIS,...,1743161021,25200,1611407,Chaiyaphum,200,2025,3,28,1,6
3,THA,16.23932,102.07933,306.22,3.16,1.68,2025-03-27,19:21:00,Aqua,MODIS,...,1743161021,25200,1611407,Chaiyaphum,200,2025,3,28,1,6
4,THA,19.45405,100.56052,310.83,2.73,1.58,2025-03-28,03:28:00,Terra,MODIS,...,1743161510,25200,1607758,Phayao,200,2025,3,28,1,6


In [20]:
dfmerge[['latitude_x','longitude_x','near_lat','near_lon','latitude_y','longitude_y']]

,latitude_x,longitude_x,near_lat,near_lon,latitude_y,longitude_y
0,16.25453,102.07751,15.8068,102.0315,15.8068,102.0315
1,17.09465,101.71175,17.2218,102.4260,17.2218,102.4260
2,16.25340,102.08438,15.8068,102.0315,15.8068,102.0315
3,16.23932,102.07933,15.8068,102.0315,15.8068,102.0315
4,19.45405,100.56052,19.2149,100.2024,19.2149,100.2024
...,...,...,...,...,...,...
2993,15.49447,104.97675,15.2284,104.8586,15.2284,104.8586
2994,15.24393,104.56009,15.1182,104.3220,15.1182,104.3220
2995,15.40026,104.56705,15.2287,104.8564,15.2287,104.8564
2996,15.60570,105.25562,15.2284,104.8586,15.2284,104.8586
